In [3]:
library(devtools)
library("fgsea")
library(devtools)
library(tidyverse)
library(Matrix)

In [4]:
read_table_run_FGSEA <- function(de_seq2_input, location, functional_annotations){
    
    input_file_name <- here(location, de_seq2_input)
    sb_denovo <- read.table(input_file_name,
                          header = TRUE)
    
    sb_denovo.selected <- sb_denovo  %>% 
        dplyr::select(gene_name, stat)

    ranks <- deframe(sb_denovo.selected)
    
    fgseaRes <- fgsea(pathways=functional_annotations, stats=ranks, 
                  nperm=10000,
                 minSize = 10, maxSize = 600)
    
    fgseaResTidy <- fgseaRes %>%
        as_tibble()
        
    
    fgseaResTidy$celltype <- de_seq2_input

    return(fgseaResTidy)
}

In [5]:
# pathways <- read_delim("/home/jpm73279/genome_downloads/Sbicolor.v5.1.prelim.annot/Sb.functional_transcripts.txt", delim = "\t" ,
#                       col_names = c("geneID", "term", "GOID"))
# pathway_format <- pathways  %>% 
#     dplyr::select(geneID, term)  %>% 
#     distinct()  %>% 
#     group_by(term)  %>% 
#     mutate(COUNTER = 1:n()) %>%
#     ungroup() %>% 
#     group_by(term)  %>% 
#     summarise(funcion = paste(geneID, collapse=","))  %>% 
#     rename(V2 = funcion)  %>% 
#     rename(V1 = term) 
    

Rows: 74877 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): geneID, term, GOID

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [7]:
# sb_functional_annotation <- convertTerms(as.data.frame(pathway_format))


In [15]:
os_func_genes <- read.table("/home/jpm73279/genome_downloads/Osativa/v7.0/Rice_markers/Os.GO_Annot.tab.gmt", sep = "\t")
#sb_functional_annotation <- convertTerms(sb_func_genes)

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”
Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“number of items read is not a multiple of the number of columns”


In [16]:
head(os_func_genes)

V1                                                            
1 regulation_of_transcription,_DNA-templated                    
2 biological_process                                            
3 regulation_of_biosynthetic_process                            
4 regulation_of_gene_expression                                 
5 regulation_of_macromolecule_biosynthetic_process              
6 regulation_of_nucleobase-containing_compound_metabolic_process
  V2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [17]:
convert_to_named_list <- function(df) {
  # Split the V2 column by comma and use V1 as the names
  named_list <- lapply(df$V2, function(x) unlist(strsplit(x, ",")))
  names(named_list) <- df$V1
  return(named_list)
}

# Assuming your input dataframe is named df
os_functional_annotation <- convert_to_named_list(os_func_genes)

In [48]:
os_de_seq2 <- c(
  "Os.de_novo.bundle_sheath.upregulated_genes.deseq2_output.tsv",
  "Os.de_novo.companion_cell.upregulated_genes.deseq2_output.tsv",
  "Os.de_novo.epidermis.upregulated_genes.deseq2_output.tsv",
  "Os.de_novo.mesophyll.upregulated_genes.deseq2_output.tsv",
  "Os.de_novo.protoderm.upregulated_genes.deseq2_output.tsv"
)

In [49]:
library("here")
working_location <- "/scratch/jpm73279/comparative_single_cell/06.de_novo_id_markers/02.de_novo_markers/Os"
all_gathered_cell_type_FGSEA <- lapply(os_de_seq2, read_table_run_FGSEA, location = working_location, functional_annotations = os_functional_annotation)




Warning message in fgsea(pathways = functional_annotations, stats = ranks, nperm = 10000, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“All values in the stats vector are greater than zero and scoreType is "std", maybe you should switch to scoreType = "pos".”
Warning message in fgsea(pathways = functional_annotations, stats = ranks, nperm = 10000, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“All values in the stats vector are greater than zero and scoreType is "std", maybe you should switch to scoreType = "pos".”
Warning message in fgsea(pathways = functional

In [50]:
os_cell_type_FGSEA <- bind_rows(all_gathered_cell_type_FGSEA)

In [51]:
fitlered_save <- os_cell_type_FGSEA  %>% 
    group_by(celltype)  %>% 
    mutate_at(c('pval'), as.numeric)  %>% 
    dplyr::filter(pval < .05) %>% 
    dplyr::arrange(pval, descending = TRUE)  %>% 
    dplyr::arrange(celltype)  %>% 
    mutate(celltype = str_replace(celltype, "Os.de_novo.", ""))  %>% 
    mutate(celltype = str_replace(celltype, ".upregulated_genes.deseq2_output.tsv", ""))  %>% 
    dplyr::select(-leadingEdge)

#write_delim(fitlered_save, "Sb.FGSEA_results.tsv", delim = "\t")

In [52]:
unique(fitlered_save$celltype)

[1] "bundle_sheath"  "companion_cell" "epidermis"      "mesophyll"

In [53]:
fitlered_save %>%
  group_by(celltype) %>%
  arrange(padj) %>%
  slice_head(n = 10)

pathway,pval,padj,ES,NES,nMoreExtreme,size,celltype
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
cellular_protein_modification_process,0.0015013281,0.053046926,0.5252588,1.995436,12,18,bundle_sheath
protein_modification_process,0.0015013281,0.053046926,0.5252588,1.995436,12,18,bundle_sheath
macromolecule_modification,0.0015013281,0.053046926,0.5252588,1.995436,12,18,bundle_sheath
phosphorus_metabolic_process,0.0096376723,0.204318653,0.5312471,1.813404,78,13,bundle_sheath
phosphate-containing_compound_metabolic_process,0.0096376723,0.204318653,0.5312471,1.813404,78,13,bundle_sheath
phosphorylation,0.0139611626,0.211411891,0.5626055,1.764565,109,10,bundle_sheath
protein_phosphorylation,0.0139611626,0.211411891,0.5626055,1.764565,109,10,bundle_sheath
cellular_macromolecule_metabolic_process,0.0194185530,0.238594638,0.4074947,1.707006,174,25,bundle_sheath
cellular_protein_metabolic_process,0.0202580353,0.238594638,0.4290860,1.710233,178,21,bundle_sheath


In [55]:
write_delim(fitlered_save, "/home/jpm73279/Mendieta_et_al_comparative_single_cell/metrics/supplament/Os.FGSEA_results.tsv", delim = "\t")